<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Библиотеки-и-функции" data-toc-modified-id="Библиотеки-и-функции-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Библиотеки и функции</a></span><ul class="toc-item"><li><span><a href="#Функции-довести-до-ума" data-toc-modified-id="Функции-довести-до-ума-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Функции довести до ума</a></span></li></ul></li><li><span><a href="#Загрузка-и-предобработка-данных" data-toc-modified-id="Загрузка-и-предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Категориальные-признаки" data-toc-modified-id="Категориальные-признаки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Категориальные признаки</a></span></li></ul></li><li><span><a href="#Объеденим-фичи" data-toc-modified-id="Объеденим-фичи-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Объеденим фичи</a></span></li></ul></div>

In [1]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


## Библиотеки и функции

In [3]:
# Импортируем библиотеки

import json
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
import scipy.stats as stats

RANDOM_STATE = 11102023

from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV,
    cross_val_score)

from sklearn.preprocessing import (
    OneHotEncoder, 
    OrdinalEncoder, 
    StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, roc_curve, RocCurveDisplay,
    f1_score, roc_auc_score, r2_score, 
    classification_report )
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay)


from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [4]:
# Make corr function

def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

# Dooing downsample function

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

# Dooing upsample function

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

# Doing function for marking cat_features

cat = ['ctg', 'flg','channel_name','src_id']
def cat_columns(columns, cat):
    cat_columns = []
    for col in columns:
        for c in cat:
            if c in col:
                cat_columns.append(col)
    return cat_columns

# Doing function for chek dubls

def dubl_list(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

# Doing function for check correlation different methods

def check_corr_method(data, method):
    """
    Функция для расчета корреляции 'kendall','pearson','spearman' всех признаков с целевой переменной.
    ---
    Принимает на вход pd.DataFrame.
    Возвращает pd.Series с значением коэффициента корреляции для каждого признака.
    """
    # чекаем корреляцию с таргетом
    correlation = data.iloc[:, :-1].corrwith(data['target'], method=method)
    
    # сортируем
    correlation = correlation.sort_values(ascending=False)
    
    return correlation

#Добавляем наглядности к корреляции
"""
    Функция для расчета корреляции 'kendall','pearson','spearman' всех признаков с целевой переменной.
    ---
    Возвращает heatmap
    """

def heatmap_corr(data, methods, a, b):
    corrmat = data.corr(method=methods, numeric_only=False)
    f, ax = plt.subplots(figsize=(a, b))
    sns.heatmap(corrmat,  square=True);

def see_anomal(data):
    for x in data:
        boxplot = plt.boxplot(data[x].values)
        plt.close('all')
        outliers = list(boxplot["fliers"][0].get_data()[1])
        df_outliers = data[data[x].isin(outliers)] 
        print("Количество аномалий: ", len(df_outliers), "стобец:", x)
        
def anomal_non_lists(data):
    nonanomal = []
    anomal = []
    for x in data:
        boxplot = plt.boxplot(data[x].values)
        plt.close('all')
        outliers = list(boxplot["fliers"][0].get_data()[1])
        df_outliers = data[data[x].isin(outliers)] 
        if len(df_outliers) > 0:
            anomal.append(x)
        else:
            nonanomal.append(x)
            
    return anomal, nonanomal
        
        
        
def read_data(left, right):

    # следующие т столбцов, которые будем считывать + target
    columns_to_read = my_data_features_num['features'][left:right].to_list() + ['target']

    # считываем столбцы в файл
    data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=columns_to_read)

    return data

def variance_filter(data, threshold=0.5):

    # считаем дисперсию для каждого признака
    variance = data.var()

    # отбираем те фичи, у которых var > threshold
    selected_features = variance[variance > threshold].index.tolist()

    # собираем в датасет
    filtered_data = data[selected_features + ['target']]
    
    return filtered_data


def corr_with_target(data):

    # чекаем корреляцию с таргетом
    correlation = data.corrwith(data['target']).abs()
    
    # сортируем
    correlation = correlation.sort_values(ascending=False)
    
    return correlation



def check_corr_biserial(data: pd.DataFrame):

    # все признаки, кроме последнего столбца с целевой переменной
    features = data.iloc[:, :-1]

    # пустой словарь для хранения результатов
    correlations = {}

    # проходимся по каждому признаку и вычисляем коэффициент корреляции
    for feature_name in features.columns:
        feature = data[feature_name]
        target = data['target']
        correlation, p_value = pointbiserialr(feature, target)
        correlations[feature_name] = correlation

    # создаем DataFrame из словаря корреляций
    correlation = pd.DataFrame.from_dict(correlations, orient='index', columns=['point_biserial_correlation'])
    
    # сортируем
    correlation = correlation.sort_values(by='point_biserial_correlation', ascending=False)
    
    return correlation


def remove_correlated_features(data, method='pearson', threshold=0.7):
    corr_matrix = data.corr(method=method).abs()
    
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
    to_drop = [column for column in upper.columns if any(upper[column] >= threshold)]
    data.drop(to_drop, axis=1, inplace=True)
    
    return data


### Функции довести до ума

## Загрузка и предобработка данных

In [5]:
#import json file with names
with open('features_types.json') as f:
    data = json.load(f)

In [6]:
# load 
data = pd.DataFrame(data.items(), columns=['features', 'types'])
data.head()

,features,types
0,markers_0_1_cnt,numeric
1,markers_1_1_cnt,numeric
2,markers_2_1_cnt,numeric
3,markers_3_1_cnt,numeric
4,markers_4_1_cnt,numeric


In [7]:
data_features = data.iloc[1850:]
data_features.types.value_counts()

numeric            847
categorical_int     61
categorical_str     18
Name: types, dtype: int64

In [8]:
# Make list with features cat, num

data_features_num = data_features[data_features['types'] == 'numeric']['features'].tolist()
data_features_cat_int = data_features[data_features['types'] == 'categorical_int']['features'].tolist()
data_features_cat_str = data_features[data_features['types'] == 'categorical_str']['features'].tolist()

In [9]:
features = data_features_num + data_features_cat_int + data_features_cat_str
features_cat = data_features_cat_int + data_features_cat_str

In [10]:
# Load data from parquet
data_num = pd.read_parquet('dataset_train.parquet', engine='fastparquet', columns=data_features_num + ['target'])

In [11]:
# See num features
data_num.head()

,issues_47_6_sum,issues_48_1_sum,issues_49_3_sum,issues_50_6_sum,issues_51_1_sum,issues_52_3_sum,issues_53_6_sum,issues_54_1_sum,issues_55_3_sum,issues_56_6_sum,...,markers_937_1_cnt,markers_938_1_cnt,markers_939_1_cnt,markers_940_1_cnt,markers_941_1_cnt,markers_942_1_cnt,markers_943_1_cnt,markers_944_1_cnt,markers_945_1_cnt,target
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.198278,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.198278,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.198278,0.691969,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.227283,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,3.518185,1.084476,1.688183,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.198278,-0.203579,-0.021226,25.247005,-0.05907,-0.037447,-0.241147,-0.118010,0.601086,0


In [12]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Columns: 848 entries, issues_47_6_sum to target
dtypes: float64(847), int32(1)
memory usage: 4.4 GB


In [13]:
data_num.isna().mean().sort_values(ascending=False).head(10)

campaigns_272_1_part    1.0
campaigns_274_3_part    1.0
campaigns_271_6_part    1.0
campaigns_270_3_part    1.0
campaigns_269_1_sum     1.0
campaigns_268_1_part    1.0
campaigns_267_6_part    1.0
campaigns_266_3_part    1.0
campaigns_265_1_sum     1.0
campaigns_264_1_part    1.0
dtype: float64

In [14]:
data_num.dropna(thresh=(len(data_num)*0.7), axis=1, inplace=True)

In [15]:
data_num = variance_filter(data_num)

In [16]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Columns: 451 entries, user_devices_22_1_cnt to target
dtypes: float64(450), int32(1)
memory usage: 2.4 GB


In [17]:
data_num.head()

,user_devices_22_1_cnt,user_devices_23_1_cnt,user_devices_24_1_cnt,user_devices_25_1_cnt,user_devices_26_1_cnt,spas_symptoms_agr_236_12_avg,spas_symptoms_agr_237_12_std,spas_symptoms_agr_238_12_sum,spas_symptoms_agr_239_1_avg,spas_symptoms_agr_240_1_std,...,markers_937_1_cnt,markers_938_1_cnt,markers_939_1_cnt,markers_940_1_cnt,markers_941_1_cnt,markers_942_1_cnt,markers_943_1_cnt,markers_944_1_cnt,markers_945_1_cnt,target
0,-0.058899,-0.009924,-0.365287,-0.005126,-0.14696,-0.025009,-0.052235,-0.025401,-0.012182,-0.020352,...,-0.198278,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,0
1,-0.058899,-0.009924,-0.365287,-0.005126,-0.14696,-0.025009,-0.052235,-0.025401,-0.012182,-0.020352,...,-0.198278,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,0
2,-0.058899,-0.009924,-0.365287,-0.005126,-0.14696,-0.025009,-0.052235,-0.025401,-0.012182,-0.020352,...,-0.198278,0.691969,-0.021226,-0.131766,-0.05907,-0.037447,-0.241147,-0.118010,-0.486011,1
3,-0.058899,-0.009924,-0.365287,-0.005126,-0.14696,-0.025009,-0.052235,-0.025401,-0.012182,-0.020352,...,0.227283,-0.203579,-0.021226,-0.131766,-0.05907,-0.037447,3.518185,1.084476,1.688183,0
4,-0.058899,-0.009924,-0.365287,-0.005126,-0.14696,-0.025009,-0.052235,-0.025401,-0.012182,-0.020352,...,-0.198278,-0.203579,-0.021226,25.247005,-0.05907,-0.037447,-0.241147,-0.118010,0.601086,0


In [18]:
data_num = remove_correlated_features(data_num)

In [19]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Columns: 227 entries, user_devices_22_1_cnt to target
dtypes: float64(226), int32(1)
memory usage: 1.2 GB


In [20]:
see_anomal(data_num)

Количество аномалий:  2956 стобец: user_devices_22_1_cnt
Количество аномалий:  129 стобец: user_devices_23_1_cnt
Количество аномалий:  121460 стобец: user_devices_24_1_cnt
Количество аномалий:  26 стобец: user_devices_25_1_cnt
Количество аномалий:  18001 стобец: user_devices_26_1_cnt
Количество аномалий:  0 стобец: spas_symptoms_agr_236_12_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_239_1_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_248_12_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_260_12_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_263_1_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_284_12_avg
Количество аномалий:  0 стобец: spas_symptoms_agr_285_12_std
Количество аномалий:  0 стобец: user_active_24_0_dt
Количество аномалий:  0 стобец: basic_info_0_0_avg
Количество аномалий:  481 стобец: user_devices_27_1_cnt
Количество аномалий:  0 стобец: campaigns_0_1_cnt
Количество аномалий:  0 стобец: campaigns_29_1_sum
Количество аномалий:  0 стобец

In [21]:
anomal_num, nonanomal_num = anomal_non_lists(data_num)
anomal_num 

['user_devices_22_1_cnt',
 'user_devices_23_1_cnt',
 'user_devices_24_1_cnt',
 'user_devices_25_1_cnt',
 'user_devices_26_1_cnt',
 'user_devices_27_1_cnt',
 'user_devices_28_1_cnt',
 'user_devices_29_1_cnt',
 'target']

In [22]:
data_num.drop(anomal_num, axis= 1, inplace= True)
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Columns: 218 entries, spas_symptoms_agr_236_12_avg to markers_945_1_cnt
dtypes: float64(218)
memory usage: 1.1 GB


### Категориальные признаки

In [55]:
# Load data from parquet
data_cat = pd.read_parquet('dataset_train.parquet', engine='fastparquet', columns=features_cat + ['target'])

In [56]:
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Data columns (total 80 columns):
 #   Column                               Non-Null Count   Dtype
---  ------                               --------------   -----
 0   communication_availability_46_1_flg  702086 non-null  int32
 1   communication_availability_47_1_flg  702086 non-null  int32
 2   communication_availability_48_1_flg  702086 non-null  int32
 3   communication_availability_49_1_flg  702086 non-null  int32
 4   communication_availability_50_1_flg  702086 non-null  int32
 5   user_active_5_1_flg                  702086 non-null  int32
 6   exist_tariff_2_1_flg                 702086 non-null  int8 
 7   deact_react_13_1_ctg                 702086 non-null  int8 
 8   tariff_plans_24_src_id               702086 non-null  int32
 9   communication_availability_51_1_flg  702086 non-null  int32
 10  communication_availability_52_1_flg  702086 non-null  int32
 11  communication_availability_53_1_flg  70

In [57]:
# Searching columns with anomal
see_anomal(data_cat)

Количество аномалий:  76341 стобец: communication_availability_46_1_flg
Количество аномалий:  141171 стобец: communication_availability_47_1_flg
Количество аномалий:  93688 стобец: communication_availability_48_1_flg
Количество аномалий:  0 стобец: communication_availability_49_1_flg
Количество аномалий:  0 стобец: communication_availability_50_1_flg
Количество аномалий:  0 стобец: user_active_5_1_flg
Количество аномалий:  0 стобец: exist_tariff_2_1_flg
Количество аномалий:  31879 стобец: deact_react_13_1_ctg
Количество аномалий:  172824 стобец: tariff_plans_24_src_id
Количество аномалий:  0 стобец: communication_availability_51_1_flg
Количество аномалий:  0 стобец: communication_availability_52_1_flg
Количество аномалий:  0 стобец: communication_availability_53_1_flg
Количество аномалий:  61835 стобец: communication_availability_54_1_flg
Количество аномалий:  81313 стобец: migrant_0_1_flg
Количество аномалий:  0 стобец: communication_availability_55_1_flg
Количество аномалий:  59747 с

In [58]:
# Dell columns with alot of Nans cels
data_cat.dropna(thresh=(len(data_cat)*0.7), axis=1, inplace=True)

In [59]:
# Chek and dell col with bigger var
data_cat = variance_filter(data_cat)

In [60]:
# Chek and dell col with correllated columns
data_cat = remove_correlated_features(data_cat, method='spearman')

In [61]:
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Data columns (total 14 columns):
 #   Column                               Non-Null Count   Dtype
---  ------                               --------------   -----
 0   tariff_plans_24_src_id               702086 non-null  int32
 1   communication_availability_56_1_ctg  702086 non-null  int32
 2   communication_availability_57_1_ctg  702086 non-null  int32
 3   social_dem_1_0_flg                   702086 non-null  int32
 4   tariff_plans_26_src_id               702086 non-null  int32
 5   materials_details_13_1_ctg           702086 non-null  int64
 6   cities_0_0_ctg                       702086 non-null  int64
 7   cities_1_0_ctg                       702086 non-null  int64
 8   materials_details_14_1_ctg           702086 non-null  int64
 9   info_house_10_0_ctg                  702086 non-null  int64
 10  agreement_type_0_0_ctg               702086 non-null  int64
 11  autopay_0_0_ctg                      70

In [62]:
# Searching columns with anomal
anomal_cat, nonanomal_cat = anomal_non_lists(data_cat)
anomal_cat

['tariff_plans_24_src_id',
 'communication_availability_57_1_ctg',
 'social_dem_1_0_flg',
 'tariff_plans_26_src_id',
 'materials_details_13_1_ctg',
 'materials_details_14_1_ctg',
 'autopay_0_0_ctg',
 'target']

In [33]:
anomal_cat_ = ['tariff_plans_24_src_id',
 'communication_availability_57_1_ctg',
 'social_dem_1_0_flg',
 'tariff_plans_26_src_id',
 'materials_details_13_1_ctg',
 'materials_details_14_1_ctg',
 'autopay_0_0_ctg']

In [34]:
data_cat.drop(anomal_cat_, axis= 1, inplace= True)
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702086 entries, 0 to 702085
Data columns (total 7 columns):
 #   Column                               Non-Null Count   Dtype
---  ------                               --------------   -----
 0   communication_availability_56_1_ctg  702086 non-null  int32
 1   cities_0_0_ctg                       702086 non-null  int64
 2   cities_1_0_ctg                       702086 non-null  int64
 3   info_house_10_0_ctg                  702086 non-null  int64
 4   agreement_type_0_0_ctg               702086 non-null  int64
 5   area_1_0_ctg                         702086 non-null  int64
 6   target                               702086 non-null  int32
dtypes: int32(2), int64(5)
memory usage: 32.1 MB


In [63]:
data_cat = data_cat.astype('category')

## Объеденим фичи 

In [64]:
data_all = data_cat.join(data_num)

In [65]:
data_all.drop_duplicates(inplace=True)

In [66]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 701733 entries, 0 to 702085
Columns: 232 entries, tariff_plans_24_src_id to markers_945_1_cnt
dtypes: category(14), float64(218)
memory usage: 1.2 GB


In [67]:
data_all['target'].value_counts()

0    696264
1      5469
Name: target, dtype: int64

In [68]:
features = data_all.drop(['target'], axis=1)
target = data_all['target']

In [69]:
cat_col = cat_columns(data_all, cat)
cat_col

['tariff_plans_24_src_id',
 'communication_availability_56_1_ctg',
 'communication_availability_57_1_ctg',
 'social_dem_1_0_flg',
 'tariff_plans_26_src_id',
 'materials_details_13_1_ctg',
 'cities_0_0_ctg',
 'cities_1_0_ctg',
 'materials_details_14_1_ctg',
 'info_house_10_0_ctg',
 'agreement_type_0_0_ctg',
 'autopay_0_0_ctg',
 'area_1_0_ctg']

In [70]:
model_lgbm = LGBMClassifier(random_state=42, categorical_feature='name:cat_col', class_weight = 'balanced')
model_lgbm.fit(features, target)

C:\Users\ipd08\anaconda3\lib\site-packages\lightgbm\basic.py:1873: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\ipd08\anaconda3\lib\site-packages\lightgbm\basic.py:1893: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=name:cat_col, categorical_column=0,1,2,3,4,5,6,7,8,9,10,11,12 will be ignored. Current value: categorical_feature=name:cat_col
[LightGBM] [Info] Number of positive: 5469, number of negative: 696264
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9777
[LightGBM] [Info] Number of data points in the train set: 701733, number of used features: 231
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(categorical_feature='name:cat_col', class_weight='balanced',
               random_state=42)

In [73]:
feature_importance = np.array(model_lgbm.feature_importances_)
feature_names = np.array(features.columns)
data = {'feature_names':feature_names,'feature_importance':feature_importance}
fitures_df = pd.DataFrame(data=data, dtype = np.int32).sort_values(by=['feature_importance'], ascending=False).head(50)
fitures_df

C:\Users\ipd08\AppData\Local\Temp\ipykernel_2480\3446683018.py:4: FutureWarning: Could not cast to int32, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  fitures_df = pd.DataFrame(data=data, dtype = np.int32).sort_values(by=['feature_importance'], ascending=False).head(50)


,feature_names,feature_importance
0,tariff_plans_24_src_id,919
7,cities_1_0_ctg,502
21,basic_info_0_0_avg,103
8,materials_details_14_1_ctg,85
20,user_active_24_0_dt,54
4,tariff_plans_26_src_id,49
92,campaigns_401_3_part,39
5,materials_details_13_1_ctg,38
190,markers_904_1_cnt,33
54,campaigns_250_1_sum,33


In [74]:
columns_to_read = fitures_df['feature_names'].to_list()

In [75]:
columns_to_read

['tariff_plans_24_src_id',
 'cities_1_0_ctg',
 'basic_info_0_0_avg',
 'materials_details_14_1_ctg',
 'user_active_24_0_dt',
 'tariff_plans_26_src_id',
 'campaigns_401_3_part',
 'materials_details_13_1_ctg',
 'markers_904_1_cnt',
 'campaigns_250_1_sum',
 'campaigns_367_3d6_part',
 'campaigns_394_3d6_part',
 'campaigns_390_1d3_part',
 'markers_905_1_cnt',
 'communication_availability_56_1_ctg',
 'markers_933_1_cnt',
 'spas_symptoms_agr_285_12_std',
 'campaigns_276_1_sum',
 'campaigns_357_1_sum',
 'campaigns_403_3d6_part',
 'markers_808_1_cnt',
 'campaigns_281_1_part',
 'markers_925_1_cnt',
 'agreement_type_0_0_ctg',
 'campaigns_385_3d6_part',
 'markers_807_1_cnt',
 'spas_symptoms_agr_284_12_avg',
 'campaigns_29_1_sum',
 'campaigns_399_1d3_part',
 'campaigns_38_1_part',
 'campaigns_363_1d3_part',
 'campaigns_359_1_sum',
 'campaigns_381_1d3_part',
 'markers_896_1_cnt',
 'campaigns_388_1_part',
 'markers_908_1_cnt',
 'markers_945_1_cnt',
 'spas_symptoms_agr_248_12_avg',
 'campaigns_0_1_cnt'